<div style="width: 100%; overflow: hidden;">
    <div style="width: 150px; float: left;"> <img src="data/D4Sci_logo_ball.png" alt="Data For Science, Inc" align="left" border="0"> </div>
    <div style="float: left; margin-left: 10px;"> <h1>LangChain for Generative AI</h1>
<h1>LangChain</h1>
        <p>Bruno Gonçalves<br/>
        <a href="http://www.data4sci.com/">www.data4sci.com</a><br/>
            @bgoncalves, @data4sci</p></div>
</div>

In [1]:
from collections import Counter
from pprint import pprint
from operator import itemgetter

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 

import torch

import openai
from openai import OpenAI

import transformers
from transformers import pipeline
from transformers import set_seed
set_seed(42) # Set the seed to get reproducible results


import langchain
from langchain.chains import create_sql_query_chain
from langchain.tools import DuckDuckGoSearchRun

import langchain_openai
from langchain_openai import ChatOpenAI

import langchain_anthropic
from langchain_anthropic import ChatAnthropic

import langchain_core
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder, PromptTemplate
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.runnables import RunnablePassthrough

import langchain_community
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_community.utilities import SQLDatabase
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool

import watermark

%load_ext watermark
%matplotlib inline

We start by print out the versions of the libraries we're using for future reference

In [2]:
%watermark -n -v -m -g -iv

Python implementation: CPython
Python version       : 3.13.3
IPython version      : 9.2.0

Compiler    : Clang 17.0.0 (clang-1700.0.13.3)
OS          : Darwin
Release     : 25.2.0
Machine     : arm64
Processor   : arm
CPU cores   : 16
Architecture: 64bit

Git hash: 59665e09cdb9e15991a6739b876a356cbd24f05a

langchain_anthropic: 0.3.14
langchain_community: 0.3.24
transformers       : 4.52.3
langchain_openai   : 0.3.18
matplotlib         : 3.10.3
langchain          : 0.3.25
watermark          : 2.5.0
langchain_core     : 0.3.62
torch              : 2.7.0
pandas             : 2.2.3
openai             : 1.78.1
numpy              : 2.2.5



Load default figure style

In [3]:
plt.style.use('d4sci.mplstyle')

# OpenAI

The first step is generate API key on the OpenAI website and store it as the "OPENAI_API_KEY" variable in your local environment. Without it we won't be able to do anything. You can find your API key in your using settings: https://help.openai.com/en/articles/4936850-where-do-i-find-my-secret-api-key. Then we are ready to instantiate the client

In [4]:
client = OpenAI()

In [ ]:
print("\n".join([model.id for model in client.models.list().data]))

In [ ]:
response = client.chat.completions.create(
  model="gpt-4o",
  messages=[
        {
            "role": "user", 
            "content": "What was Superman's weakness?"
        },
    ]
)

In [ ]:
print(response)

In [ ]:
print(response.choices[0].message.content)

# LangChain

We instantiate the LangChain interface for OpenAI

In [ ]:
model = ChatOpenAI(model="gpt-4o")

In [ ]:
messages = [
    SystemMessage(content="What was Superman's weakness?"),
]

output = model.invoke(messages)
print(output)

In [ ]:
output.response_metadata["token_usage"]

In [ ]:
parser = StrOutputParser()

In [ ]:
result = model.invoke(messages)

In [ ]:
parser.invoke(result)

Let us create our first chain. Stages of the chain are conencted with the pipe '|' character

In [ ]:
chain = model | parser

Now whenver we call __invoke()__ on the chain, it automatically runs all the steps

In [ ]:
chain.invoke(messages)

We can also create templates for our prompts, following conventions similar to the Jinja templating system

In [ ]:
system_template = "Translate the following text into {language}:"

And we can combine multiple messages into a single template

In [ ]:
prompt_template = ChatPromptTemplate.from_messages(
    [
     ("system", system_template), 
     ("user", "{text}")
    ]
)

To instantiate the prompt, we must provide the correct fields

In [ ]:
result = prompt_template.invoke(
    {
        "language": "italian", 
        "text": "Be the change that you wish to see in the world."
    }
)

result

The full interaction is:

In [ ]:
result.to_messages()

In [ ]:
chain = prompt_template | model | parser

In [ ]:
chain.invoke({
    "language": "italian", 
    "text": "Be the change that you wish to see in the world."
})

# Anthropic

In [ ]:
?ChatAnthropic

In [ ]:
model_a = ChatAnthropic(model="claude-haiku-4-5-20251001")

In [ ]:
model_a.invoke("What is Superman's weakness?")

In [ ]:
chain_a = prompt_template | model_a | parser

In [ ]:
model_a

In [ ]:
chain_a.invoke(
    {
        "language": "italian", 
        "text": "Be the change that you wish to see in the world."
    }
)

# Message History

In [ ]:
store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [ ]:
with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [ ]:
config = {"configurable": {"session_id": "abc2"}}

In [ ]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi! I'm Bruno")],
    config=config,
)

response.content

In [ ]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

In [ ]:
store

In [ ]:
config = {"configurable": {"session_id": "abc3"}}

response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

In [ ]:
config = {"configurable": {"session_id": "abc2"}}

response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [ ]:
chain = prompt | model | parser

In [ ]:
response = chain.invoke({"messages": [HumanMessage(content="hi! I'm bob")]})

response

In [ ]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history)

In [ ]:
config = {"configurable": {"session_id": "abc5"}}

In [ ]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi! I'm Jim")],
    config=config,
)

response

# Database Integration

In [ ]:
db = SQLDatabase.from_uri("sqlite:///data/Northwind_small.sqlite")

In [ ]:
print(db.dialect)

In [ ]:
print(db.get_usable_table_names())

In [ ]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

In [ ]:
write_query = create_sql_query_chain(llm, db)

In [ ]:
response = write_query.invoke({"question": "How many customers are there"}) 
response

In [ ]:
db.run(response)

In [ ]:
execute_query = QuerySQLDataBaseTool(db=db)

In [ ]:
sql_chain = write_query | execute_query

In [ ]:
sql_chain.invoke({"question": "How many employees are there"})

In [ ]:
answer_prompt = PromptTemplate.from_template(
    """Given the following user question, corresponding SQL query, and SQL result, answer the user question.

Question: {question}
SQL Query: {query}
SQL Result: {result}
Answer: """
)

answer = answer_prompt | llm | StrOutputParser()
chain = (
    RunnablePassthrough.assign(query=write_query).assign(
        result=itemgetter("query") | execute_query
    )
    | answer
)

chain.invoke({"question": "How many employees are there"})

In [ ]:
RunnablePassthrough.assign(query=write_query).invoke({"question": "How many employees are there"})

In [ ]:
RunnablePassthrough.assign(query=write_query).assign(
        result=itemgetter("query")).invoke({"question": "How many employees are there"})

In [ ]:
search = DuckDuckGoSearchRun()
search.run("When will the next solar eclipse be?")

<center>
     <img src="data/D4Sci_logo_full.png" alt="Data For Science, Inc" align="center" border="0" width=300px> 
</center>